This code takes the reprojected carbon stock layers and calculates zonal statistics across the y2y region based on protected area status.

In [105]:
# import packages
import geopandas as gpd
import numpy as np
import rioxarray as rxr
import xarray

import pandas as pd
from pandas.api.types import is_numeric_dtype

from geocube.api.core import make_geocube

from shapely.validation import make_valid

import warnings

In [106]:
# load wdpa and wdoecm layers clipped to y2y and reproject to match rasters
protect = gpd.read_file(
    './land_cover/protected_areas_WDPA_WDOECM_clean.shp').to_crs(
        '+proj=laea +lat_0=55 +lon_0=-125 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs')

# make sure geometries are valid
protect['geometry'] = protect['geometry'].apply(make_valid)

In [107]:
# explore protected layer
protect.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1444 entries, 0 to 1443
Data columns (total 35 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   WDPAID    1444 non-null   float64 
 1   WDPA_PI   1444 non-null   object  
 2   PA_DEF    1444 non-null   object  
 3   NAME      1444 non-null   object  
 4   ORIG_NA   1444 non-null   object  
 5   DESIG     1444 non-null   object  
 6   DESIG_E   1444 non-null   object  
 7   DESIG_T   1444 non-null   object  
 8   IUCN_CA   1444 non-null   object  
 9   INT_CRI   1444 non-null   object  
 10  MARINE    1444 non-null   object  
 11  REP_M_A   1444 non-null   float64 
 12  GIS_M_A   1444 non-null   float64 
 13  REP_ARE   1444 non-null   float64 
 14  GIS_ARE   1444 non-null   float64 
 15  NO_TAKE   1444 non-null   object  
 16  NO_TK_A   0 non-null      float64 
 17  STATUS    1444 non-null   object  
 18  STATUS_   1304 non-null   float64 
 19  GOV_TYP   1444 non-null   object  
 20  

In [108]:
# unique gov_types
protect['GOV_TYP'].unique()

array(['Federal or national ministry or agency',
       'Sub-national ministry or agency', 'Collaborative governance',
       'Indigenous peoples', 'Non-profit organisations',
       'Joint governance', 'Individual landowners', 'Not Reported'],
      dtype=object)

In [109]:
# total gis area grouped by gov type
protect.groupby('GOV_TYP')['GIS_ARE'].sum().sort_values(ascending=False)

GOV_TYP
Federal or national ministry or agency    150059.058994
Sub-national ministry or agency           125092.962143
Collaborative governance                   67487.958495
Not Reported                               42030.616323
Non-profit organisations                    3295.843628
Indigenous peoples                           513.253207
Joint governance                              78.814716
Individual landowners                          1.006097
Name: GIS_ARE, dtype: float64

In [110]:
# total area based on polygon projection grouped by gov type
# these numbers significantly less than GIS_AREA...
# probably because now there are no overlapping areas
protect['PROJ_AREA'] = protect.geometry.area
protect.groupby('GOV_TYP')['PROJ_AREA'].sum().sort_values(
    ascending=False) / 1e+6  # km^2

GOV_TYP
Federal or national ministry or agency    128912.341998
Sub-national ministry or agency            99449.935921
Collaborative governance                   59988.055007
Non-profit organisations                    3219.344672
Indigenous peoples                           513.101881
Not Reported                                 116.560538
Joint governance                              78.839974
Individual landowners                          1.003462
Name: PROJ_AREA, dtype: float64

In [111]:
# total area grouped by owner type
protect.groupby(['OWN_TYP', 'GOV_TYP'])['PROJ_AREA'].sum(
).sort_values(ascending=False) / 1e+6  # km^2

OWN_TYP                   GOV_TYP                               
State                     Federal or national ministry or agency    126542.324819
                          Sub-national ministry or agency            99180.157978
Multiple ownership        Collaborative governance                   30990.940738
State                     Collaborative governance                   15748.797873
Joint ownership           Collaborative governance                   13248.316396
Individual landowners     Federal or national ministry or agency      2360.045205
                          Non-profit organisations                    1755.481931
Non-profit organisations  Non-profit organisations                    1463.862741
Not Reported              Indigenous peoples                           513.101881
                          Sub-national ministry or agency              212.811357
                          Not Reported                                 116.560538
Joint ownership           Joint g

In [112]:
# filter by columns needed
protect = protect.filter(
    ['NAME', 'GOV_TYP', 'PROJ_AREA', 'geometry'])
protect

,NAME,GOV_TYP,PROJ_AREA,geometry
0,Tepee Creek,Federal or national ministry or agency,2.494700e+06,"POLYGON ((597263.966 -663064.788, 597294.672 -..."
1,Cliff Lake,Federal or national ministry or agency,9.183715e+06,"POLYGON ((1062018.96 -1037304.218, 1062044.283..."
2,Canyon Creek,Federal or national ministry or agency,3.610998e+06,"POLYGON ((611714.174 -702940.612, 611750.824 -..."
3,Little Pend Oreille National Wildlife Refu,Federal or national ministry or agency,6.470890e+05,"POLYGON ((545524.502 -698300.829, 545530.046 -..."
4,Little Pend Oreille National Wildlife Refu,Federal or national ministry or agency,3.312820e+05,"POLYGON ((548968.95 -689252.658, 548967.777 -6..."
...,...,...,...,...
1439,Blackhawk on the River,Non-profit organisations,2.177625e+05,"MULTIPOLYGON (((700543.984 -1083430.879, 70056..."
1440,Boone Family Trust,Non-profit organisations,2.223795e+05,"POLYGON ((682436.064 -1056212.274, 682408.247 ..."
1441,Busby,Non-profit organisations,3.861260e+05,"POLYGON ((709639.402 -1082659.862, 709671.199 ..."
1442,Driscoll Ranch - Meadow,Non-profit organisations,2.741246e+06,"POLYGON ((665114.253 -1117126.527, 665118.953 ..."


In [113]:
# rename gov types columns
rename_gov_dict = {'Indigenous peoples': 'Collaborative and/or Indigenous', 
                   'Collaborative governance': 'Collaborative and/or Indigenous', 
                   'Joint governance': 'Collaborative and/or Indigenous',
                   'Individual landowners': 'Collaborative and/or Indigenous'}

protect['GOV_TYP'] = protect['GOV_TYP'].replace(rename_gov_dict)
protect

,NAME,GOV_TYP,PROJ_AREA,geometry
0,Tepee Creek,Federal or national ministry or agency,2.494700e+06,"POLYGON ((597263.966 -663064.788, 597294.672 -..."
1,Cliff Lake,Federal or national ministry or agency,9.183715e+06,"POLYGON ((1062018.96 -1037304.218, 1062044.283..."
2,Canyon Creek,Federal or national ministry or agency,3.610998e+06,"POLYGON ((611714.174 -702940.612, 611750.824 -..."
3,Little Pend Oreille National Wildlife Refu,Federal or national ministry or agency,6.470890e+05,"POLYGON ((545524.502 -698300.829, 545530.046 -..."
4,Little Pend Oreille National Wildlife Refu,Federal or national ministry or agency,3.312820e+05,"POLYGON ((548968.95 -689252.658, 548967.777 -6..."
...,...,...,...,...
1439,Blackhawk on the River,Non-profit organisations,2.177625e+05,"MULTIPOLYGON (((700543.984 -1083430.879, 70056..."
1440,Boone Family Trust,Non-profit organisations,2.223795e+05,"POLYGON ((682436.064 -1056212.274, 682408.247 ..."
1441,Busby,Non-profit organisations,3.861260e+05,"POLYGON ((709639.402 -1082659.862, 709671.199 ..."
1442,Driscoll Ranch - Meadow,Non-profit organisations,2.741246e+06,"POLYGON ((665114.253 -1117126.527, 665118.953 ..."


In [114]:
# recheck gov type
protect.groupby('GOV_TYP')['PROJ_AREA'].sum().sort_values(
    ascending=False) / 1e+6  # km^2

GOV_TYP
Federal or national ministry or agency    128912.341998
Sub-national ministry or agency            99449.935921
Collaborative and/or Indigenous            60581.000325
Non-profit organisations                    3219.344672
Not Reported                                 116.560538
Name: PROJ_AREA, dtype: float64

In [115]:
# check not reported gov_type
protect.query("GOV_TYP=='Not Reported'")

,NAME,GOV_TYP,PROJ_AREA,geometry
1325,Creston Valley,Not Reported,2.067170e+07,"MULTIPOLYGON (((611854.81 -618892.197, 611845...."
1346,Nahanni National Park,Not Reported,7.097790e+06,"POLYGON ((75313.304 690326.695, 73069.591 6785..."
1347,Yellowstone National Park,Not Reported,3.569361e+06,"MULTIPOLYGON (((1090407.374 -1001914.465, 1090..."
1348,Canadian Rocky Mountain Parks,Not Reported,8.093714e+07,"MULTIPOLYGON (((401759.54 -220001.846, 401779...."
1349,Waterton Glacier International Peace Park,Not Reported,4.284553e+06,"MULTIPOLYGON (((789792.989 -639129.18, 789823...."


In [116]:
# change gov_type for not reported values
protect.loc[1325, 'GOV_TYP'] = 'Sub-national ministry or agency'
protect.loc[1346, 'GOV_TYP'] = 'Federal or national ministry or agency'
protect.loc[1347, 'GOV_TYP'] = 'Federal or national ministry or agency'
protect.loc[1348, 'GOV_TYP'] = 'Federal or national ministry or agency'
protect.loc[1349, 'GOV_TYP'] = 'Federal or national ministry or agency'

In [117]:
# check value
protect.loc[1325, ['NAME', 'GOV_TYP']]

NAME                        Creston Valley
GOV_TYP    Sub-national ministry or agency
Name: 1325, dtype: object

In [118]:
# load y2y polygon and set to same crs as protect
y2y = gpd.read_file('./study_area/Y2Y_RegionBoundary_Final2013/Y2Y_RegionBoundary.shp').to_crs(protect.crs)

# take the geometric difference to get a polygon for all non-protected area
y2y_nonprot = y2y.union_all().difference(protect.union_all())

In [119]:
# add polygon for non-protected areas to protect
# initialize as gpd
new_row = gpd.GeoDataFrame({'geometry': [y2y_nonprot], 
                            'NAME': 'Non-protected', 
                            'GOV_TYP': 'Non-protected', 
                            'PROJ_AREA': y2y_nonprot.area}, 
                            crs=protect.crs)

# add to protect
protect = pd.concat([protect, new_row], ignore_index=True)

In [120]:
# recheck gov type
protect.groupby('GOV_TYP')['PROJ_AREA'].sum().sort_values(
    ascending=False) / 1e+6  # km^2

GOV_TYP
Non-protected                             1.077671e+06
Federal or national ministry or agency    1.290082e+05
Sub-national ministry or agency           9.947061e+04
Collaborative and/or Indigenous           6.058100e+04
Non-profit organisations                  3.219345e+03
Name: PROJ_AREA, dtype: float64

In [121]:
# define function to extract zonal stats
def extract_stats (dat_fp, dat_name, vector, vect_var, stat):

    # Suppress UserWarning within this function
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", UserWarning)

        # create new vect variable so don't edit data in place
        vect = vector.copy()

        # check if vect_var column is numeric
        numeric = True
        if not is_numeric_dtype(vect[vect_var]):
            vect['key'] = pd.factorize(vect[vect_var])[0]
            vect.rename(columns={vect_var: 'orig_name', 'key': vect_var}, inplace=True)
            numeric = False

        # clip data to vector layer
        dat = rxr.open_rasterio(dat_fp, masked=True
                                ).rio.clip(vect.geometry.values, vect.crs, from_disk=True)
        dat.name = dat_name

        # create output grid
        out_grid = make_geocube(
            vector_data=vect,
            measurements=[vect_var],
            like=dat
        )

        # merge the datacube with the data
        out_grid[dat_name] = (dat.dims, dat.values,
                                dat.attrs, dat.encoding)
        
        # group data by vector variable
        grouped = out_grid.drop_vars(['spatial_ref']).groupby(vect_var)

        # calculate stats
        if stat == 'sum':
            table = grouped.sum()
        if stat == 'mean':
            table = grouped.mean()

        # reset indicies and drop band
        table = table.to_dataframe()
        table.reset_index(level='band', drop=True, inplace=True)

        # reset keys to original values
        if numeric == False:
            mapping = dict(zip(vect[vect_var], vect['orig_name']))
            table.index = table.index.map(mapping)

        # return table
        return table

In [122]:
# load carbon rasters in format for zonal stats function
# file paths
carbon_fp = ['./carbon_stock_data/output_layers/carbon_sothe_spawn_t_laea.tif',
             './carbon_stock_data/output_layers/soc_0_1m_t_laea.tif',
             './carbon_flux_data/output_layers/emissions_gfw_t_yr_laea.tif',
             './carbon_flux_data/output_layers/removals_gfw_t_yr_laea.tif',
             './carbon_stock_data/output_layers/carbon_sothe_spawn_t_ha_laea.tif',            
             './carbon_stock_data/output_layers/soc_0_1m_t_ha_laea.tif',
             './carbon_flux_data/output_layers/emissions_gfw_t_ha_laea.tif',
             './carbon_flux_data/output_layers/removals_gfw_t_ha_laea.tif']

carbon_names = ['carbon_t',
                'soc_t',
                'emissions_t_yr',
                'removals_t_yr',
                'carbon_t_ha',
                'soc_t_ha',
                'emissions_t_ha',
                'removals_t_ha']

stat_names = ['sum',
              'sum',
              'sum',
              'sum',
              'mean',
              'mean',
              'mean',
              'mean']

In [124]:
# extract zonal stats
for i in range(len(carbon_fp)):
    if i == 0:
        stats = extract_stats(dat_fp = carbon_fp[i], dat_name = carbon_names[i], vector=protect, vect_var='GOV_TYP', stat=stat_names[i])
    else:
        stats = pd.concat([stats,
                          extract_stats(dat_fp = carbon_fp[i], dat_name = carbon_names[i], vector=protect, vect_var='GOV_TYP', stat=stat_names[i])],
                          axis=1)
        
stats

/Users/bermane/Team Braintree Dropbox/Ethan Berman/Python Projects/y2y-carbon/.venv/lib/python3.12/site-packages/xarray/core/concat.py:540: UserWarning: No index created for dimension GOV_TYP because variable GOV_TYP is not a coordinate. To create an index for GOV_TYP, please first call `.set_coords('GOV_TYP')` on this object.
  ds.expand_dims(dim_name, create_index_for_new_dim=create_index_for_new_dim)
/Users/bermane/Team Braintree Dropbox/Ethan Berman/Python Projects/y2y-carbon/.venv/lib/python3.12/site-packages/xarray/core/concat.py:540: UserWarning: No index created for dimension GOV_TYP because variable GOV_TYP is not a coordinate. To create an index for GOV_TYP, please first call `.set_coords('GOV_TYP')` on this object.
  ds.expand_dims(dim_name, create_index_for_new_dim=create_index_for_new_dim)
/Users/bermane/Team Braintree Dropbox/Ethan Berman/Python Projects/y2y-carbon/.venv/lib/python3.12/site-packages/xarray/core/concat.py:540: UserWarning: No index created for dimension GO

,carbon_t,soc_t,emissions_t_yr,removals_t_yr,carbon_t_ha,soc_t_ha,emissions_t_ha,removals_t_ha
GOV_TYP,,,,,,,,
Federal or national ministry or agency,5.033212e+08,2.771852e+09,2.539669e+06,3.959882e+06,39.868315,219.876764,54.319250,13.524230
Sub-national ministry or agency,4.454234e+08,2.345764e+09,1.059341e+06,6.076531e+06,46.317794,244.197615,57.625693,22.817708
Collaborative and/or Indigenous,1.082693e+08,1.532786e+09,5.666325e+05,8.710108e+05,18.342451,259.879802,66.292074,10.709372
Non-profit organisations,9.512454e+06,5.353432e+07,5.611170e+04,1.080022e+05,29.617765,166.893808,89.545490,23.425149
Non-protected,4.483550e+09,2.165785e+10,2.937447e+07,5.156292e+07,42.514836,205.670349,85.619673,18.054609


In [141]:
# add area to table
new_row = pd.DataFrame({'GOV_TYP': protect.groupby('GOV_TYP')['PROJ_AREA'].sum().index.values,
                        'area_km2': protect.groupby('GOV_TYP')['PROJ_AREA'].sum().values / 1e+6})
new_row.set_index('GOV_TYP', inplace=True)

# concat to stats
stats = pd.concat([stats, new_row], axis=1)
stats

,carbon_t,soc_t,emissions_t_yr,removals_t_yr,carbon_t_ha,soc_t_ha,emissions_t_ha,removals_t_ha,area_km2
GOV_TYP,,,,,,,,,
Federal or national ministry or agency,5.033212e+08,2.771852e+09,2.539669e+06,3.959882e+06,39.868315,219.876764,54.319250,13.524230,1.290082e+05
Sub-national ministry or agency,4.454234e+08,2.345764e+09,1.059341e+06,6.076531e+06,46.317794,244.197615,57.625693,22.817708,9.947061e+04
Collaborative and/or Indigenous,1.082693e+08,1.532786e+09,5.666325e+05,8.710108e+05,18.342451,259.879802,66.292074,10.709372,6.058100e+04
Non-profit organisations,9.512454e+06,5.353432e+07,5.611170e+04,1.080022e+05,29.617765,166.893808,89.545490,23.425149,3.219345e+03
Non-protected,4.483550e+09,2.165785e+10,2.937447e+07,5.156292e+07,42.514836,205.670349,85.619673,18.054609,1.077671e+06


In [145]:
# export to csv
stats.to_excel(
    './outputs/y2y_carbon_protected_area.xlsx', index=True)